In [ ]:
import polars as pl


In [ ]:
jan_2017_to_now_df = pl.read_csv(
    "./datasets/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv",
    dtypes={"floor_area_sqm": pl.Float32, "resale_price": pl.Float32},
)

In [ ]:
jan_2017_to_now_df = jan_2017_to_now_df.with_columns(
    remaining_lease_years=pl.col("remaining_lease")
    .str.split(" ")
    .list.get(0)
    .cast(pl.Int64),
)

In [ ]:
jan_2017_to_now_df

In [ ]:
jan_1990_dec_2016_df = pl.read_csv(
    "./resale-flat-prices-jan-1990-dec-2016.csv",
    dtypes={"floor_area_sqm": pl.Float32, "resale_price": pl.Float32, "block": pl.Utf8},
)

In [ ]:
jan_1990_dec_2016_df

In [ ]:
jan_1990_now_df = pl.concat([jan_1990_dec_2016_df, jan_2017_to_now_df], how="diagonal")
jan_1990_now_df = jan_1990_now_df.sort("month")
jan_1990_now_df

In [ ]:
jan_1990_now_df = jan_1990_now_df.with_columns(
    block_street_name=pl.col("block") + " " + pl.col("street_name")
)
jan_1990_now_df

In [ ]:
unique_block_street_name = jan_1990_now_df.select(["block", 'street_name']).unique()
unique_block_street_name.sort(['block', 'street_name'])

In [ ]:
blocks_from_csv = pl.read_csv("./blocks_from_resale.csv")
blocks_from_csv

In [ ]:
new_blocks = unique_block_street_name.join(blocks_from_csv, how="anti", on=["block", "street_name"])
new_blocks

In [ ]:
import requests

In [ ]:
def get_latlong(search_val):
    params = {"searchVal": search_val, "returnGeom": "Y", "getAddrDetails": "Y"}
    r = requests.get(f"https://onemap.gov.sg/api/common/elastic/search", params=params)
    resp = r.json()
    result = resp["results"][0]
    return {'latitude': float(result["LATITUDE"]), 'longitude': float(result["LONGITUDE"])}

In [ ]:
new_blocks_lat_long = []
errors = []

for row in new_blocks.iter_rows(named=True):
    search_val = f"{row['block']} {row['street_name']}"
    try:
        lat_long = get_latlong(search_val)
        row_lat_long = {**row, **lat_long}
        new_blocks_lat_long.append(row_lat_long) 
    except Exception as e:
        print(e)
        errors.append(search_val, e)

print(errors)
new_blocks_lat_long

In [ ]:
new_blocks_lat_long_df = pl.DataFrame(new_blocks_lat_long)
new_blocks_lat_long_df = new_blocks_lat_long_df.with_columns(
    block_street_name=pl.col("block") + " " + pl.col("street_name")
)
new_blocks_lat_long_df = new_blocks_lat_long_df.drop(["block", "street_name"])
new_blocks_lat_long_df

In [ ]:
resale_blocks_geocode_df = pl.read_csv("./resale_blocks_geocode_block_street_name.csv")
resale_blocks_geocode_df

In [ ]:
new_resale_blocks_geocode_df = pl.concat([resale_blocks_geocode_df, new_blocks_lat_long_df], how="diagonal")
new_resale_blocks_geocode_df

In [ ]:
new_resale_blocks_geocode_df.write_csv("./new_resale_blocks_geocode.csv")